In [12]:
import os   #helps me erase a warning that i am gettin
import numpy as np  #the data are numpy when i first get them
from sklearn.preprocessing import StandardScaler #i will use this to scale the data
from sklearn.decomposition import PCA #i will use this to reduce the dimensionality of the data
from sklearn.cluster import KMeans #i will use this to cluster the data


os.environ["LOKY_MAX_CPU_COUNT"] = "8"  # This line erases a warning that i am getting

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] #the names of the classes      

file_names = []
files = []
#create X_train, y_train, X_test, y_test
X_train = np.full((50000,3072),0,dtype=int)
X_test = np.full((10000,3072),0,dtype=int)
y_train = np.full((50000,),0,dtype=int)
y_test = np.full((10000,),0,dtype=int)

# the unpickle() function loads the CIFAR-10 data from the file
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Creating a list with the names of the files
for i in range(1,6):
    file = f"data_batch_{i}"
    file_names.append(file)
file_names.append("test_batch")

#storing the unpickled dictionaries to a list
for file in file_names:
    cifar10_dict = unpickle(file)
    files.append(cifar10_dict)

# Splitting the data to train and test set
# Printing the shapes to make sure that everything went well
i=0
for file in files:
    print(f"Append: {file[b'batch_label']}")
    if file[b'batch_label'] != b'testing batch 1 of 1':
        #this is the X_train, y_train
        X_train[i*10000:(i+1)*10000,:] = file[b'data']
        y_train[i*10000:(i+1)*10000] = file[b'labels'] 
        i+=1
       
    else:
        #i have just finished X_train, y_train   
        #this is the X_test, y_test
        X_test[:,:] = file[b'data']
        y_test[:] = file[b'labels']
        print(f"Shape X_train: {X_train.shape}\nShape y_train: {y_train.shape}")
        print(f"Shape X_test: {X_test.shape}\nShape y_test: {y_test.shape}\n")

Append: b'training batch 1 of 5'
Append: b'training batch 2 of 5'
Append: b'training batch 3 of 5'
Append: b'training batch 4 of 5'
Append: b'training batch 5 of 5'
Append: b'testing batch 1 of 1'
Shape X_train: (50000, 3072)
Shape y_train: (50000,)
Shape X_test: (10000, 3072)
Shape y_test: (10000,)



In [18]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

class mRBF:
    def __init__(self, sigma, n_clusters):
        self.sigma = sigma
        self.n_clusters = n_clusters
        self.kmeans = None
        self.cluster_centers_ = None
        self.cluster_labels_ = None

    def fit(self, X_train, y_train):
        # Perform k-means clustering to find cluster centers
        self.kmeans = KMeans(n_clusters=self.n_clusters, random_state=0).fit(X_train)
        self.cluster_centers_ = self.kmeans.cluster_centers_
        self.cluster_labels_ = self.kmeans.labels_
        self.y_train = y_train
        self.sigma = dmax / np.sqrt(2 * self.n_clusters)

    def radial_basis_function(self, x_input, center):
        return np.exp(-np.linalg.norm(x_input - center) ** 2 / (2 * self.sigma ** 2))
    
    def approximating_function(self, x_input,y_test):
        # Compute the RBF outputs for each cluster center
        rbf_outputs = np.array([self.radial_basis_function(x_input, center) for center in self.cluster_centers_])
        
        # Initialize an array to store the sum of RBF outputs for each class
        class_scores = np.zeros(len(np.unique(self.y_train)))
        
        # Sum the RBF outputs for each class
        for i, score in enumerate(rbf_outputs):
            class_scores[y_test[i]] += score
        
        # Choose the class with the highest score
        predicted_class = np.argmax(class_scores)
        return predicted_class





In [8]:
model = mRBF(1)

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

pca = PCA(n_components=0.9)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


In [10]:
print(f"X_train_pca shape: {X_train_pca.shape}")
print(f"X_test_pca shape: {X_test_pca.shape}")

X_train_pca shape: (50000, 103)
X_test_pca shape: (10000, 103)


In [ ]:
# Example usage
sigma = 1
n_clusters = 10
model = mRBF(sigma, n_clusters)



# Fit the model
model.fit(X_train_pca, y_train)

# Predict the class for each sample in X_test_pca
predicted_classes = [model.approximating_function(x,y_test) for x in X_test_pca]
acc = accuracy_score(y_test, predicted_classes)
print(f"Accuracy: {acc}")

Accuracy: 0.1239


In [25]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import cdist

class mRBF:
    def __init__(self, n_clusters,sigma = None):
        self.sigma = sigma
        self.n_clusters = n_clusters
        self.kmeans = None
        self.cluster_centers_ = None
        self.cluster_labels_ = None

    def fit(self, X_train, y_train):
        # Perform k-means clustering to find cluster centers
        self.kmeans = KMeans(n_clusters=self.n_clusters, random_state=0).fit(X_train)
        self.cluster_centers_ = self.kmeans.cluster_centers_
        self.cluster_labels_ = self.kmeans.labels_
        self.y_train = y_train

        # Calculate the maximum distance between cluster centers
        pairwise_distances = cdist(self.cluster_centers_, self.cluster_centers_, 'euclidean')
        dmax = np.max(pairwise_distances)
        
        # Set sigma using the maximum distance
        self.sigma = dmax / np.sqrt(2 * self.n_clusters)

    def radial_basis_function(self, x_input, center):
        return np.exp(-np.linalg.norm(x_input - center) ** 2 / (2 * self.sigma ** 2))
    
    def approximating_function(self, x_input):
        # Compute the RBF outputs for each cluster center
        rbf_outputs = np.array([self.radial_basis_function(x_input, center) for center in self.cluster_centers_])
        
        # Initialize an array to store the sum of RBF outputs for each class
        class_scores = np.zeros(len(np.unique(self.y_train)))
        
        # Sum the RBF outputs for each class
        for i, score in enumerate(rbf_outputs):
            # Find the class label for the cluster center
            cluster_label = self.cluster_labels_[i]
            class_label = self.y_train[cluster_label]
            class_scores[class_label] += score
        
        # Choose the class with the highest score
        predicted_class = np.argmax(class_scores)
        return predicted_class

    def predict(self, X_test):
        # Predict the class for each sample in X_test
        predictions = np.array([self.approximating_function(x_input) for x_input in X_test])
        return predictions

# Example usage
n_clusters = 20
model = mRBF(n_clusters)



# Fit the model
model.fit(X_train_pca, y_train)

# Predict the classes for the test set
predictions = model.predict(X_test_pca)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.0927
